In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [6]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band4
  atmpro = mls
  band = 4
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 980
  vmin = 800
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.001170                    1 -0.001010
0.000750        2 -0.001170                    2 -0.001030
0.001052        3 -0.001171                    3 -0.001005
0.001476        4 -0.001171                    4 -0.001002
0.002070        5 -0.001169                    5 -0.001002
0.002904        6 -0.001166                    6 -0.000999
0.004074        7 -0.001162                    7 -0.000997
0.005714        8 -0.001157                    8 -0.000992
0.008015        9 -0.001150                    9 -0.000987
0.011243       10 -0.001143                   10 -0.000983
0.015771       11 -0.001133                   11 -0.000982
0.022122       12 -0.001123                   12 -0.000979
0.031031       13 -0.001111                   13 -0.000973
0.043528       14 -0.001096                   14 -0.000965
0.061057       15 -0.001038                   15 -0.000931
0.085645       16 -0.000772                   16 -0.000756
0.120136       17 -0.000011                   17 -0.000240
0.168516       18  0.001465                   18  0.000778
0.236378       19  0.004017                   19  0.002534
0.331549       20  0.008145                   20  0.005358
0.465100       21  0.014507                   21  0.009772
0.652400       22  0.023918                   22  0.016928
0.915100       23  0.033992                   23  0.024751
1.283650       24  0.033935                   24  0.025161
1.800600       25  0.024437                   25  0.018721
2.525700       26  0.015515                   26  0.012338
3.542800       27  0.009099                   27  0.007463
4.969550       28  0.004839                   28  0.004044
6.970850       29  0.002189                   29  0.001766
9.778100       30  0.000558                   30  0.000260
13.715850      31 -0.000486                   31 -0.000762
19.239350      32 -0.001128                   32 -0.001416
26.987250      33 -0.001465                   33 -0.001784
37.855300      34 -0.001653                   34 -0.002003
53.100050      35 -0.001801                   35 -0.002154
73.887500      36 -0.001924                   36 -0.002259
97.662500      37 -0.002033                   37 -0.002344
121.437500     38 -0.002141                   38 -0.002402
145.212500     39 -0.002244                   39 -0.002393
168.987500     40 -0.002358                   40 -0.002447
192.762500     41 -0.002515                   41 -0.002543
216.537500     42 -0.002651                   42 -0.002764
240.312500     43 -0.002684                   43 -0.002873
264.087500     44 -0.002601                   44 -0.002841
287.862500     45 -0.002397                   45 -0.002672
311.637500     46 -0.002067                   46 -0.002378
335.412500     47 -0.001608                   47 -0.001918
359.187500     48 -0.001020                   48 -0.001342
382.962500     49 -0.000303                   49 -0.000638
406.737500     50  0.000539                   50  0.000205
430.512500     51  0.001497                   51  0.001167
454.287500     52  0.002560                   52  0.002244
478.062500     53  0.003718                   53  0.003442
501.837500     54  0.004966                   54  0.004757
525.612500     55  0.006305                   55  0.006116
549.387500     56  0.007739                   56  0.007592
573.162500     57  0.009269                   57  0.009192
596.937500     58  0.010898                   58  0.010920
620.712500     59  0.012627                   59  0.012780
644.487500     60  0.014455                   60  0.014708
668.262500     61  0.016381                   61  0.016708
692.037500     62  0.018405                   62  0.018827
715.812500     63  0.020528                   63  0.021071
739.587500     64  0.022747                   64  0.023431
763.362500     65  0.025056                   65  0.025897
78

# Fluxes by Layer

CRD                          CLIRAD             \
                        flug          fldg      fnetg      i       flug   
pressure    level                                                         
0.000000    1     -60.933315  0.000000e+00 -60.933315     11 -60.961173   
0.000624    2     -60.933316  6.326707e-08 -60.933315     11 -60.961173   
0.000876    3     -60.933316  8.919601e-08 -60.933316     11 -60.961173   
0.001229    4     -60.933316  1.269363e-07 -60.933316     11 -60.961173   
0.001723    5     -60.933316  1.820057e-07 -60.933316     11 -60.961174   
0.002417    6     -60.933316  2.623295e-07 -60.933316     11 -60.961174   
0.003391    7     -60.933316  3.794871e-07 -60.933316     11 -60.961174   
0.004757    8     -60.933317  5.503525e-07 -60.933316     11 -60.961174   
0.006672    9     -60.933317  7.994991e-07 -60.933316     11 -60.961175   
0.009359    10    -60.933318  1.162758e-06 -60.933317     11 -60.961175   
0.013128    11    -60.933319  1.692323e-06 -60.933317     11 -60.961176   
0.018415    12    -60.933320  2.464241e-06 -60.933318     11 -60.961177   
0.025830    13    -60.933322  3.589291e-06 -60.933319     11 -60.961179   
0.036232    14    -60.933325  5.228871e-06 -60.933320     11 -60.961181   
0.050823    15    -60.933330  7.623952e-06 -60.933322     11 -60.961185   
0.071291    16    -60.933336  1.140637e-05 -60.933325     11 -60.961190   
0.100000    17    -60.933346  1.864097e-05 -60.933327     11 -60.961198   
0.140271    18    -60.933362  3.415725e-05 -60.933327     11 -60.961211   
0.196760    19    -60.933385  6.722455e-05 -60.933318     11 -60.961230   
0.275997    20    -60.933416  1.366501e-04 -60.933280     11 -60.961257   
0.387100    21    -60.933453  2.799701e-04 -60.933173     11 -60.961289   
0.543100    22    -60.933475  5.708163e-04 -60.932905     11 -60.961318   
0.761700    23    -60.933432  1.147270e-03 -60.932285     11 -60.961315   
1.068500    24    -60.933233  2.183535e-03 -60.931050     11 -60.961227   
1.498800    25    -60.932910  3.589420e-03 -60.929320     11 -60.961093   
2.102400    26    -60.932631  5.058101e-03 -60.927573     11 -60.961041   
2.949000    27    -60.932487  6.470210e-03 -60.926017     11 -60.961145   
4.136600    28    -60.932534  7.797443e-03 -60.924737     11 -60.961458   
5.802500    29    -60.932828  9.046462e-03 -60.923782     11 -60.962034   
8.139200    30    -60.933432  1.025622e-02 -60.923176     11 -60.962938   
11.417000   31    -60.934433  1.147381e-02 -60.922959     11 -60.964262   
16.014700   32    -60.935951  1.272720e-02 -60.923224     11 -60.966126   
22.464000   33    -60.938134  1.404831e-02 -60.924086     11 -60.968683   
31.510500   34    -60.941206  1.554994e-02 -60.925656     11 -60.972187   
44.200100   35    -60.945537  1.739683e-02 -60.928140     11 -60.977058   
62.000000   36    -60.951631  1.969353e-02 -60.931937     11 -60.983853   
85.775000   37    -60.959793  2.243569e-02 -60.937357     11 -60.992896   
109.550000  38    -60.968170  2.508770e-02 -60.943082     11 -61.002099   
133.325000  39    -60.976928  2.781489e-02 -60.949113     11 -61.011516   
157.100000  40    -60.986034  3.060221e-02 -60.955432     11 -61.020890   
180.875000  41    -60.995650  3.357825e-02 -60.962071     11 -61.030566   
204.650000  42    -61.006434  3.727959e-02 -60.969154     11 -61.041090   
228.425000  43    -61.019009  4.238965e-02 -60.976619     11 -61.053566   
252.200000  44    -61.033472  4.929365e-02 -60.984178     11 -61.068041   
275.975000  45    -61.049772  5.826826e-02 -60.991504     11 -61.084361   
299.750000  46    -61.067831  6.957746e-02 -60.998254     11 -61.102388   
323.525000  47    -61.087551  8.347723e-02 -61.004074     11 -61.122063   
347.300000  48    -61.108818  1.002145e-01 -61.008604     11 -61.142992   
371.075000  49    -61.131502  1.200249e-01 -61.011477     11 -61.165167   
394.850000  50    -61.155461  1.431293e-01 -61.012332     11 -61.188588   
418.625000  51    -61.180531  1.697168e-01 -61.010

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')